# Looking at https://github.com/ytsvetko/metaphor

In [502]:
##all the imports...
#%pylab inline
import re
import math
import string
import nltk
import pprint
import matplotlib
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk import bigrams
from nltk import collocations
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.corpus import brown

## Start by loading the .csv files and setting the columns.
### For these .csv files, downloaded from https://github.com/ytsvetko/metaphor inputs and saved as .csv with the first row added as "sample"

In [180]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet.head()

,sample
0,angry welt
1,bald assertion
2,bare outline
3,black humor
4,blind alley


In [181]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet['metaphor'] = 1
df_anmet['an'] = 1
df_anmet['svo'] = 0
df_anmet['metanet'] = 0
df_anmet.head()
len(df_anmet)

100

In [182]:
df_annonmet = pd.read_csv("an_nonmets.csv", low_memory=False)
df_annonmet['metaphor'] = 0
df_annonmet['an'] = 1
df_annonmet['svo'] = 0
df_annonmet['metanet'] = 0
df_annonmet.head()
len(df_annonmet)

100

In [183]:
df_svomet = pd.read_csv("svo_mets.csv", low_memory=False)
df_svomet['metaphor'] = 1
df_svomet['an'] = 0
df_svomet['svo'] = 1
df_svomet['metanet'] = 0
df_svomet.head()
len(df_svomet)

111

In [184]:
df_svononmet = pd.read_csv("svo_nonmets.csv", low_memory=False)
df_svononmet['metaphor'] = 0
df_svononmet['an'] = 0
df_svononmet['svo'] = 1
df_svononmet['metanet'] = 0
df_svononmet.head()
len(df_svononmet)

111

In [185]:
df_metanet = pd.read_csv("metanet.csv", low_memory=False)
df_metanet['metaphor'] = 1
df_metanet['an'] = 0
df_metanet['svo'] = 0
df_metanet['metanet'] = 1
df_metanet.head()
# len(df_metanet)

,sample,metaphor,an,svo,metanet
0,ability to evaluate government is ability to see,1,0,0,1
1,ability to evaluate is ability to see,1,0,0,1
2,ability to know is ability to see,1,0,0,1
3,abusive political leaders are physical bullies,1,0,0,1
4,accepting is swallowing,1,0,0,1


In [186]:
#combine into one df
frames = [df_anmet, df_annonmet, df_svomet, df_svononmet, df_metanet]
df_combo = pd.concat(frames)
df_combo.reset_index(drop=True, inplace=True)
df_combo.shape
# df_combo.replace(to_replace='none', value="")
# #         tokenizer = RegexpTokenizer(r'\w+')
# #         word_list = tokenizer.tokenize(line)
# #         filtered_words = [word for word in word_list if word not in stopwords.words('english')]
# df_combo.replace(to_replace='is', value="")
stop = ['*none', '.', 'is']
df_combo['sample'].apply(lambda x: [item for item in x if item not in stop])
df_combo[200:500]

,sample,metaphor,an,svo,metanet
200,conversation turn subject,1,0,1,0
201,resumption bring relief,1,0,1,0
202,economy move direction,1,0,1,0
203,service meet expectation,1,0,1,0
204,material live dream,1,0,1,0
205,unemployment stand *none,1,0,1,0
206,action talk *none,1,0,1,0
207,statement sit *none,1,0,1,0
208,income fall *none,1,0,1,0
209,silence speak volume,1,0,1,0


## Break down into training, dev, and test sets

In [187]:
random_index = np.random.permutation(df_combo.index)
df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled = df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled.reset_index(drop=True, inplace=True)
len(df_shuffled)

1606

In [188]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)
#train_size = round(rows*.6)
train_size = round(rows*.9)
#dev_size   = round(rows*.2)
dev_size   = round(rows*.1)
df_train = df_shuffled.loc[:train_size]
df_train.shape
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
df_dev.shape
df_test = df_shuffled.loc[dev_size+train_size:].reset_index(drop=True)
df_test.shape

Rows: 1606
Columns: 5


(0, 5)

## CountVectorizer
### Use count vecotrizer with df = 3 for unigram and bigrams to predict dev set

In [195]:
vocab = list(set(brown.words()))

In [196]:
# vec = CountVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b', analyzer=u'word', min_df=1, vocabulary=vocab)
# df_train = df_train.fillna("")
# df_dev = df_dev.fillna("")
# df_test = df_test.fillna("")


vec = TfidfVectorizer(ngram_range=(2, 3), token_pattern=r'\b\w+\b', analyzer=u'char', min_df=1)
df_train = df_train.fillna("")
df_dev = df_dev.fillna("")
df_test = df_test.fillna("")

In [197]:
arr_train_feature_sparse = vec.fit_transform(df_train['sample'])
arr_train_feature_sparse
arr_train_feature = arr_train_feature_sparse.toarray()
feature_labels = vec.get_feature_names()

In [198]:
arr_dev_feature_sparse = vec.transform(df_dev["sample"])
arr_dev_feature = arr_dev_feature_sparse.toarray()

In [199]:
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor']) #defining features (from reviews) and passing in Category label
logreg_predictions = logreg_model.predict(arr_dev_feature)
accuracy_score(df_dev['metaphor'], logreg_predictions)

0.88198757763975155

In [200]:
feature_sum = arr_train_feature.sum(axis=0)   #sum the counts of each feature

df_feature_sum = pd.DataFrame({'counts': feature_sum})
df_feature_sum.index = vec.get_feature_names()
df_feature_sum.sort('counts', ascending=False)[0:10]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,counts
s,55.478196
in,54.085359
i,51.684293
is,50.882446
a,49.849230
ti,49.823581
on,49.616411
is,45.124110
is,44.978664
ng,41.920643


In [322]:
## make a file of sample sentences to practice on
## loop through sentence and .split()
## use nltk bigrams
##
## put into pandas as df_test

In [201]:
word_list = 'Jamal was a pig at dinner'.split()
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
filtered_words
testmeta = list(trigrams(filtered_words))
testmeta
testmeta1 = [' '.join(x) for x in testmeta]
testmeta1
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample']
print(filtered_words)

['Jamal', 'pig', 'dinner']


In [202]:
arr_test_feature_sparse = vec.transform(df_test['sample'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
logreg_predictions

array([1])

In [203]:
df_test = pd.DataFrame(testmeta1)
df_test

,0
0,Jamal pig dinner


In [204]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
#         word_list = line.split()
        tokenizer = RegexpTokenizer(r'\w+')
        word_list = tokenizer.tokenize(line)
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
#         testmeta1.append(' '.join(filtered_words))
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
#         testmeta1.append(' '.join(x) for x in filtered_words)
# print(testmeta1)

df_test =pd.DataFrame(testmeta1)
df_test
df_test = df_test.fillna("")
df_test
# df_test.columns = ['sample']
# df_test = pd.DataFrame(testmeta1)
# df_test
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
# df_test = df_test.fillna("")
# df_test

In [216]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor0'] = logreg_predictions
if df_test['sample0'] == '':
    df_test['metaphor0'] = 0
else:
    df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor1'] = logreg_predictions
if df_test['sample1'] == '':
    df_test['metaphor1'] = 0
else:
    df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor2'] = logreg_predictions
if df_test['sample2'] == '':
    df_test['metaphor2'] = 0
else:
    df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor3'] = logreg_predictions
if df_test['sample3'] == '':
    df_test['metaphor3'] = 0
else:
    df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor4'] = logreg_predictions
if df_test['sample4'] == '':
    df_test['metaphor4'] = 0
else:
    df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor5'] = logreg_predictions
if df_test['sample5'] == '':
    df_test['metaphor5'] = 0
else:
    df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
# df_test['metaphor6'] = logreg_predictions
if df_test['sample6'] == '':
    df_test['metaphor6'] = 0
else:
    df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor0'] + df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] + df_test['metaphor6'] 

# def count_metaphors():
#     if sum(df_test['sum_metaphor']) >= 1:
#         df_test['sum_metaphor'] = 1
#     else:
#         df_test['sum_metaphor'] = 0
# count_metaphors()

df_test

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [249]:
print(df_test['sample3'] == '')

0      True
1      True
2      True
3      True
4     False
5      True
6     False
7      True
8     False
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16    False
17     True
18    False
19    False
20     True
21     True
22     True
23     True
24    False
25     True
26    False
27    False
28     True
29     True
      ...  
41     True
42     True
43     True
44     True
45     True
46     True
47    False
48    False
49     True
50     True
51    False
52     True
53     True
54     True
55     True
56     True
57     True
58    False
59     True
60     True
61     True
62    False
63    False
64    False
65     True
66    False
67     True
68     True
69    False
70    False
Name: sample3, dtype: bool


NoneType

In [229]:
df_test['sample3'].str.len()

0      0
1      0
2      0
3      0
4     10
5      0
6     13
7      0
8      9
9      0
10     0
11     0
12     0
13     0
14     0
15    13
16    12
17     0
18    13
19    14
20     0
21     0
22     0
23     0
24    11
25     0
26    14
27    13
28     0
29     0
      ..
41     0
42     0
43     0
44     0
45     0
46     0
47    13
48    10
49     0
50     0
51    11
52     0
53     0
54     0
55     0
56     0
57     0
58    11
59     0
60     0
61     0
62    12
63    17
64    14
65     0
66    10
67     0
68     0
69    11
70    14
Name: sample3, dtype: int64

In [280]:
with open("an_nonmets.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0']
df_test = df_test.fillna("")
df_test

arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

sum(df_test['metaphor0'])
#len(df_test['metaphor0'])

3

In [308]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
# df_test = df_test.ix[:, 0:6]
# df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
# df_test = df_test.fillna("")
df_test

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across,arranged across different,across different regions,different regions organ.,...,None,None,None,None,None,None,None,None,None,None
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues,rainbow hues individual,hues individual words,individual words concepts,...,concepts convey grouped,convey grouped together,grouped together clumps,together clumps white,clumps white matter.,None,None,None,None,None
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect,specific aspect language,aspect language represented,"language represented brain,",...,"brain, case semantics,","case semantics, meanings","semantics, meanings words,”","meanings words,” said","words,” said Jack","said Jack Gallant,","Jack Gallant, neuroscientist","Gallant, neuroscientist University","neuroscientist University California,","University California, Berkeley."
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;,life love; death,love; death taxes;,"death taxes; clouds,",...,"clouds, Florida bra.",None,None,None,None,None,None,None,None,None
7,All light networks.,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas","study, atlas demonstrates",atlas demonstrates modern,demonstrates modern imaging,...,imaging transform knowledge,transform knowledge brain,knowledge brain performs,brain performs important,performs important tasks.,None,None,None,None,None
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [80]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
sum(df_test['sum_metaphor'])
#len(df_test['sum_metaphor'])
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6,metaphor0,metaphor1,metaphor2,metaphor3,metaphor4,metaphor5,metaphor6,sum_metaphor
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across,1,1,1,1,1,1,1,1
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues,1,1,1,1,1,1,1,1
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect,1,1,1,1,1,1,1,1
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,,1,1,1,1,1,1,1,1
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,,1,1,1,1,1,1,1,1
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,,1,1,1,1,1,1,1,1
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;,1,1,1,1,1,1,1,1
7,All light networks.,,,,,,,1,1,1,1,1,1,1,1
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas",1,1,1,1,1,1,1,1
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,,1,1,1,1,1,1,1,1


In [101]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [103]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

df_test

,sample0,metaphor0
0,scientists created “atlas brain” reveals meani...,1
1,"like colourful quilt laid cortex, atlas displa...",1
2,“our goal build giant atlas shows one specific...,1
3,no single brain region holds one word concept.,1
4,a single brain spot associated number related ...,1
5,and single word lights many different brain sp...,1
6,together make networks represent meanings word...,1
7,all light networks.,1
8,described “tour de force” one researcher invol...,1
9,"with advances, technology could profound impac...",1


In [81]:
df_test.ix[:, 0:5].head()

,sample0,sample1,sample2,sample3,sample4
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays"
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one
3,No single brain,single brain region,brain region holds,region holds one,holds one word
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related


In [179]:
len(df_test.columns)
#df_test[0]

6

In [306]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [104]:
import nltk, re
from nltk.corpus import brown
from nltk import word_tokenize

In [105]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2


def train_tagger(already_tagged_sents):
    train_sents, test_sents = create_data_sets(already_tagged_sents)
    ngram_tagger = build_backoff_tagger(train_sents)
    print ("%0.3f pos accuracy on test set" % ngram_tagger.evaluate(test_sents))
    return ngram_tagger


In [106]:
def train_tagger_on_brown():
    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance','science_fiction'])
    return train_tagger(brown_tagged_sents)


In [107]:
def tokenize_text(corpus):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences    
    return [nltk.word_tokenize(word) for word in raw_sents]

def create_corpus(f):
    with open(f, 'r') as text_file:
        new_corpus = text_file.read()
    return new_corpus


In [108]:
brown_tagger = train_tagger_on_brown()

0.911 pos accuracy on test set


In [176]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

[[('scientists', 'NNS'),
  ('have', 'HV'),
  ('created', 'VBN'),
  ('an', 'AT'),
  ('atlas', 'NN'),
  ('of', 'IN'),
  ('the', 'AT'),
  ('brain', 'NN'),
  ('that', 'CS'),
  ('reveals', 'VBZ'),
  ('how', 'WRB'),
  ('the', 'AT'),
  ('meanings', 'NNS'),
  ('of', 'IN'),
  ('words', 'NNS'),
  ('are', 'BER'),
  ('arranged', 'VBN'),
  ('across', 'IN'),
  ('different', 'JJ'),
  ('regions', 'NNS'),
  ('of', 'IN'),
  ('the', 'AT'),
  ('organ', 'NN')],
 [('like', 'CS'),
  ('a', 'AT'),
  ('colourful', 'NN'),
  ('quilt', 'NN'),
  ('laid', 'VBN'),
  ('over', 'IN'),
  ('the', 'AT'),
  ('cortex', 'NN'),
  ('the', 'AT'),
  ('atlas', 'NN'),
  ('displays', 'VBZ'),
  ('in', 'IN'),
  ('rainbow', 'NN'),
  ('hues', 'NNS'),
  ('how', 'WRB'),
  ('individual', 'JJ'),
  ('words', 'NNS'),
  ('and', 'CC'),
  ('the', 'AT'),
  ('concepts', 'NNS'),
  ('they', 'PPSS'),
  ('convey', 'VB'),
  ('can', 'MD'),
  ('be', 'BE'),
  ('grouped', 'VBN'),
  ('together', 'RB'),
  ('in', 'IN'),
  ('clumps', 'NNS'),
  ('of', 'IN'),
  

In [177]:
def train_tagger_on_brown_augmented_with_additional_sents():

    additional_sents = [[('colorful', 'JJ'), ('quilt', 'NN')],
                        [('regions', 'NN'), ('represent', 'VB'), ('world', 'NN')],
                        [('public', 'NN'), ('explore', 'VB'), ('brain', 'NN')]]


    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance', 'science_fiction'])
    
    #append hand-tagged cooking sentences to the front of the training data
    all_tagged_sents = additional_sents + brown_tagged_sents
    return train_tagger(all_tagged_sents)

brown_and_additional_tagger = train_tagger_on_brown_augmented_with_cooking_sents()

0.911 pos accuracy on test set


In [173]:
better_sentences = get_pos(testmeta1, brown_and_additional_tagger)

In [178]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

[[('the', 'AT'),
  ('snow', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('white', 'JJ'),
  ('blanket', 'NN')],
 [('the', 'AT'),
  ('hospital', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('refrigerator', 'NN')],
 [('the', 'AT'),
  ('classroom', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('zoo', 'NN')],
 [('america', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('melting', 'VBG'),
  ('pot', 'NN')],
 [('her', 'PP$'),
  ('lovely', 'JJ'),
  ('voice', 'NN'),
  ('was', 'BEDZ'),
  ('music', 'NN'),
  ('to', 'IN'),
  ('his', 'PP$'),
  ('ears', 'NNS')],
 [('life', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('roller', 'NN'),
  ('coaster', 'NN')],
 [('the', 'AT'),
  ('alligator', 'NN'),
  ('s', 'NN'),
  ('teeth', 'NNS'),
  ('are', 'BER'),
  ('white', 'JJ'),
  ('daggers', 'NN')],
 [('their', 'PP$'),
  ('home', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('prison', 'NN')],
 [('the', 'AT'),
  ('slide', 'NN'),
  ('on', 'IN'),
  ('the', 'AT'),
  ('playground', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('hot', 'J

# Switch to Classification Features with NLTK Classification Code

In [286]:
%matplotlib inline 
import nltk
from nltk.corpus import names
import random

In [288]:
f = open('an_mets.txt', 'r')
x = f.readlines()
an_mets = [t.rstrip() for t in x[1::]]
f = open('an_nonmets.txt', 'r')
x = f.readlines()
an_nonmets = [t.rstrip() for t in x[1::]]
f = open('svo_mets.txt', 'r')
x = f.readlines()
svo_mets = [t.rstrip() for t in x[1::]]
f = open('svo_nonmets.txt', 'r')
x = f.readlines()
svo_nonmets = [t.rstrip() for t in x[1::]]

In [397]:
# def double_letter(word):
#     letter_list = []
#     for letter in word:
#         if word.count(letter) > 1:
#             letter_list.append(True)
#         else:
#             letter_list.append(False)
#     if True in letter_list:
#         return True
#     else:
#         return False
    
def metaphor_features(word):
    features = {}
    word = word.lower()
    features['POS'] = word[0]
#     features['last'] = word[-1]
#     features['last 2'] = word[-2]
#     features['first 3'] = word[:3]
#     features['first'] = word[:1]
#     features['length'] = len(word)
#     features['starts with K'] = word.startswith('k')
#     features['ends with i'] = word.endswith('i')
#     features['ends with a'] = word.endswith('a')
#     features['double letter'] = double_letter(word)
    return features
print(str(metaphor_features('Michelle')))  

{'POS': 'm'}


In [412]:
def create_an_data():
    an_mets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'metaphor') for line in an_mets]
    an_nonmets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'NOT metaphor') for line in an_nonmets]
    svo_mets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'metaphor') for line in svo_mets]
    svo_nonmets_tag = [(re.sub(r'[^\w\s]','',line.lower()), 'NOT metaphor') for line in svo_nonmets]
    all_an = an_mets_tag + an_nonmets_tag + svo_mets_tag + svo_nonmets_tag
    
    # Randomize the order of male and female names, and de-alphabatize
    random.shuffle(all_an)
    return all_an

all_an_data = create_an_data()
# print(all_an_data)

In [413]:
# This function allows experimentation with different feature definitions
# items is a list of (key, value) pairs from which features are extracted and training sets are made
# Feature sets returned are dictionaries of features

# This function also optionally returns the names of the training, development, 
# and test data for the purposes of error checking

def create_training_sets (feature_function, items, return_items=False):
    # Create the features sets.  Call the function that was passed in.
    # For names data, key is the name, and value is the gender
    featuresets = [(feature_function(key), value) for (key, value) in items]
    
    # Divided training and testing in thirds.  Could divide in other proportions instead.
    third = int(float(len(featuresets)) / 3.0)
    
    train_set, dev_set, test_set = featuresets[0:third], featuresets[third:third*2], featuresets[third*2:]
    train_items, dev_items, test_items = items[0:third], items[third:third*2], items[third*2:]
    if return_items == True:
        return train_set, dev_set, test_set, train_items, dev_items, test_items
    else:
        return train_set, dev_set, test_set

In [414]:
# note that this passes in a function name as an argument (gender_features)

train_set, dev_set, test_set = create_training_sets(metaphor_features, all_an_data)
cl = nltk.NaiveBayesClassifier.train(train_set)

In [415]:
print ("'white blanket': " + cl.classify(metaphor_features('white blanket')))
print ("'snow white': " + cl.classify(metaphor_features('snow white')))
print ("'white as snow': " + cl.classify(metaphor_features('white as snow')))
print ("'happy customer': " + cl.classify(metaphor_features('happy customer')))
print ("'boiling rage': " + cl.classify(metaphor_features('boiling rage')))

'white blanket': NOT metaphor
'snow white': metaphor
'white as snow': NOT metaphor
'happy customer': NOT metaphor
'boiling rage': NOT metaphor


In [416]:
testsample = 'one specific aspect'
print (testsample + ": " + cl.classify(metaphor_features(testsample)))

one specific aspect: NOT metaphor


In [417]:
print ("%.3f" % nltk.classify.accuracy(cl, dev_set))
cl.show_most_informative_features(15)

0.464
Most Informative Features
                     POS = 'n'            NOT me : metaph =      3.3 : 1.0
                     POS = 'a'            metaph : NOT me =      3.3 : 1.0
                     POS = 'f'            metaph : NOT me =      2.9 : 1.0
                     POS = 'c'            metaph : NOT me =      2.1 : 1.0
                     POS = 'p'            NOT me : metaph =      1.9 : 1.0
                     POS = 'h'            NOT me : metaph =      1.3 : 1.0
                     POS = 'm'            NOT me : metaph =      1.3 : 1.0
                     POS = 's'            metaph : NOT me =      1.2 : 1.0
                     POS = 'w'            NOT me : metaph =      1.2 : 1.0
                     POS = 't'            NOT me : metaph =      1.1 : 1.0
                     POS = 'i'            metaph : NOT me =      1.1 : 1.0
                     POS = 'r'            metaph : NOT me =      1.1 : 1.0
                     POS = 'b'            metaph : NOT me =      1.1

In [418]:

with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word.lower() not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
print(testmeta)

[['scientists created atlas', 'created atlas brain', 'atlas brain reveals', 'brain reveals meanings', 'reveals meanings words', 'meanings words arranged', 'words arranged across', 'arranged across different', 'across different regions', 'different regions organ'], ['like colourful quilt', 'colourful quilt laid', 'quilt laid cortex', 'laid cortex atlas', 'cortex atlas displays', 'atlas displays rainbow', 'displays rainbow hues', 'rainbow hues individual', 'hues individual words', 'individual words concepts', 'words concepts convey', 'concepts convey grouped', 'convey grouped together', 'grouped together clumps', 'together clumps white', 'clumps white matter'], ['our goal build', 'goal build giant', 'build giant atlas', 'giant atlas shows', 'atlas shows one', 'shows one specific', 'one specific aspect', 'specific aspect language', 'aspect language represented', 'language represented brain', 'represented brain case', 'brain case semantics', 'case semantics meanings', 'semantics meanings w

In [390]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word.lower() not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
print(testmeta)

[['snow white blanket'], [], [], ['america melting pot'], ['lovely voice music', 'voice music ears'], ['life roller coaster'], ['alligators teeth white', 'teeth white daggers'], [], ['slide playground hot', 'playground hot stove'], ['heart cold iron'], [], [], [], [], ['toms eyes ice'], ['detectives face wood', 'face wood listened', 'wood listened story'], ['feels life fashion', 'life fashion show'], [], ['kids room disaster', 'room disaster area'], ['children flowers grown', 'flowers grown concrete', 'grown concrete gardens'], ['kisses flowers affection'], ['words cotton candy'], ['marys eyes fireflies'], ['johns suggestion bandaid'], ['cast broken leg', 'broken leg plaster', 'leg plaster shackle'], ['janes ambitions house', 'ambitions house cards'], ['long hair flowing', 'hair flowing golden', 'flowing golden river'], ['computers school old', 'school old dinosaurs'], ['laughter music soul'], [], [], ['falling snowflakes dancers'], ['new haircut sheepdog'], ['five oclock interstate', 

In [419]:
metaphor_count = 0
nonmetaphor_count = 0
for x in testmeta:
    for y in x:
        testsample = y
        print (testsample + ": " + cl.classify(metaphor_features(testsample)))
        if cl.classify(metaphor_features(testsample)) == 'metaphor':
            metaphor_count += 1
        elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
            nonmetaphor_count += 1
            
print('metaphor_count:' + str(metaphor_count))
print('nonmetaphor_count:' + str(nonmetaphor_count))

scientists created atlas: metaphor
created atlas brain: metaphor
atlas brain reveals: metaphor
brain reveals meanings: NOT metaphor
reveals meanings words: NOT metaphor
meanings words arranged: NOT metaphor
words arranged across: NOT metaphor
arranged across different: metaphor
across different regions: metaphor
different regions organ: NOT metaphor
like colourful quilt: NOT metaphor
colourful quilt laid: metaphor
quilt laid cortex: NOT metaphor
laid cortex atlas: NOT metaphor
cortex atlas displays: metaphor
atlas displays rainbow: metaphor
displays rainbow hues: NOT metaphor
rainbow hues individual: NOT metaphor
hues individual words: NOT metaphor
individual words concepts: NOT metaphor
words concepts convey: NOT metaphor
concepts convey grouped: metaphor
convey grouped together: metaphor
grouped together clumps: NOT metaphor
together clumps white: NOT metaphor
clumps white matter: metaphor
our goal build: NOT metaphor
goal build giant: NOT metaphor
build giant atlas: NOT metaphor
gia

In [445]:
with open("sciencearticle.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    para_index = 0
#     text = ''.join(testset.readlines())
#     sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
#     sentences = [t.rstrip() for t in sentences]
#     print(sentences)
    for line in testset:
        para_index += 1
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
        print(para_index)
# print(testmeta)

metaphor_count = 0
nonmetaphor_count = 0
metaphor_list = []
for x in testmeta:
    for y in x:
        testsample = y
        print (testsample + ": " + cl.classify(metaphor_features(testsample)))
        if cl.classify(metaphor_features(testsample)) == 'metaphor':
            metaphor_count += 1
            metaphor_list.append(testsample)
        elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
            nonmetaphor_count += 1
            
print('metaphor_count:' + str(metaphor_count))
print('nonmetaphor_count:' + str(nonmetaphor_count))
print(metaphor_list)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
scientists created atlas: metaphor
created atlas brain: metaphor
atlas brain reveals: metaphor
brain reveals meanings: NOT metaphor
reveals meanings words: NOT metaphor
meanings words arranged: NOT metaphor
words arranged across: NOT metaphor
arranged across different: metaphor
across different regions: metaphor
different regions organ: NOT metaphor
like colourful quilt: NOT metaphor
colourful quilt laid: metaphor
quilt laid cortex: NOT metaphor
laid cortex atlas: NOT metaphor
cortex atlas displays: metaphor
atlas displays rainbow: metaphor
displays rainbow hues: NOT metaphor
rainbow hues individual: NOT metaphor
hues individual words: NOT metaphor
individual words concepts: NOT metaphor
words concepts convey: NOT metaphor
concepts convey grouped: metaphor
convey grouped together: metaphor
grouped together clumps: NOT metaphor
together clumps white: NOT metaphor
clumps white matter: metaphor
our goal build: N

In [482]:
with open("sciencearticle.txt", "r") as testset:
    word_list = ''
    highlight_bucket = []
    for line in testset:
        print(line)
        for trigram in metaphor_list:
            trigram_list = trigram.split()
            index = line.find(trigram_list[0])
            index_len = len(trigram)
            highlight_bucket.append(line[index:index+index_len])
print(highlight_bucket)

Scientists have created an “atlas of the brain” that reveals how the meanings of words are arranged across different regions of the organ.



Like a colourful quilt laid over the cortex, the atlas displays in rainbow hues how individual words and the concepts they convey can be grouped together in clumps of white matter.



“Our goal was to build a giant atlas that shows how one specific aspect of language is represented in the brain, in this case semantics, or the meanings of words,” said Jack Gallant, a neuroscientist at the University of California, Berkeley.



No single brain region holds one word or concept. A single brain spot is associated with a number of related words. And each single word lights up many different brain spots. Together they make up networks that represent the meanings of each word we use: life and love; death and taxes; clouds, Florida and bra. All light up their own networks.



Described as a “tour de force” by one researcher who was not involved in the stu

In [462]:
sample = ['the', 'methods', 'i', 'am', 'michelle']
# word_list
word_str = ' '.join(word_list)
# print(word_str)
for word in sample:
    print(word_str.find(word))

55
181
5
105
-1


In [508]:

stopwords_regex = "("
# print(stopwords.words('english'))
for word in stopwords.words('english'):
    stopwords_regex = stopwords_regex + word + "|"
stopwords_regex = stopwords_regex[:-1] + ")"
print(stopwords_regex)
punctuation_regex = r"[!\"#$%&'\(\)*+,-./:;<=>?@^_`{}~]*"

# 


(i|me|my|myself|we|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|their|theirs|themselves|what|which|who|whom|this|that|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|s|t|can|will|just|don|should|now|d|ll|m|o|re|ve|y|ain|aren|couldn|didn|doesn|hadn|hasn|haven|isn|ma|mightn|mustn|needn|shan|shouldn|wasn|weren|won|wouldn)


In [512]:
with open("sciencearticle.txt", "r") as testset:
    aritcle = testset.read().lower()
    word_list = ''
    highlight_bucket = []
    tag_list = []
    
    for trigram in metaphor_list:
#         print(trigram)
        trigram_list = trigram.split(" ")
        regex_string = trigram_list[0] + punctuation_regex + stopwords_regex + punctuation_regex + trigram_list[1] + punctuation_regex + stopwords_regex + punctuation_regex + trigram_list[2] 
        print(regex_string)
        match = re.search(regex_string, aritcle)
        complete_trigram = match.group(0)
        print(trigram, complete_trigram)
        index = aritcle.find(complete_trigram)
        print(index, trigram_list[0])
        index_len = len(trigram)
        highlight_bucket.append(aritcle[index:index+index_len])
        tag_list.append((index, index_len))
        aritcle = aritcle[index+index_len:]
    
#     for line in testset:
# #         print(line)
#         for trigram in metaphor_list:
#             trigram_list = trigram.split()
#             index = line.find(trigram_list[0])
#             index_len = len(trigram)
#             highlight_bucket.append(line[index:index+index_len])
#             line = line[index+index_len:]
print(highlight_bucket)
len(highlight_bucket)

scientists[!\"#$%&'\(\)*+,-./:;<=>?@^_`{}~]*(i|me|my|myself|we|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|their|theirs|themselves|what|which|who|whom|this|that|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|s|t|can|will|just|don|should|now|d|ll|m|o|re|ve|y|ain|aren|couldn|didn|doesn|hadn|hasn|haven|isn|ma|mightn|mustn|needn|shan|shouldn|wasn|weren|won|wouldn)[!\"#$%&'\(\)*+,-./:;<=>?@^_`{}~]*created[!\"#$%&'\(\)*+,-./:;<=>?@^_`{}~]*(i|me|my|myself|we|our|ours|ourselves|you|your|yours|yourself|yourselves|he|him|his|himself|she|her|hers|herself|it|its|itself|they|them|

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
with open("sciencearticle.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    word_list = ''
    para_index = 0
#     text = ''.join(testset.readlines())
#     sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
#     sentences = [t.rstrip() for t in sentences]
#     print(sentences)
    for line in testset:
        para_index += 1
        word_list = line.split()
        filtered_words = [re.sub(r'[^\w\s]','',word.lower()) for word in word_list if word not in stopwords.words('english')]
        testmeta.append([' '.join(x) for x in trigrams(filtered_words)])
        print(para_index)
# print(testmeta)

metaphor_count = 0
nonmetaphor_count = 0
metaphor_list = []
for x in testmeta:
    for y in x:
        testsample = y
        print (testsample + ": " + cl.classify(metaphor_features(testsample)))
        if cl.classify(metaphor_features(testsample)) == 'metaphor':
            metaphor_count += 1
            metaphor_list.append(testsample)
        elif cl.classify(metaphor_features(testsample)) == 'NOT metaphor':
            nonmetaphor_count += 1
            
print('metaphor_count:' + str(metaphor_count))
print('nonmetaphor_count:' + str(nonmetaphor_count))
print(metaphor_list)